## cls 확률화

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import timm
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

# --- [설정] ---
CONFIG = {
    'base_dir': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection',
    'manifest_path': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\det_train_manifest_2019_3ch.csv',
    'golden_bbox_csv': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\filtered_train_bounding_boxes.csv',
    'model_weight_path': r'./runs/rsna_clf_efficientnet_golden_235/best_efficientnet_golden_235.pth',
    'model_name': 'tf_efficientnetv2_s.in21k_ft_in1k',
    'img_size': 512,
    'batch_size': 32,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'output_csv': './runs/rsna_clf_efficientnet_golden_235/val_full_predictions.csv' # 👈 여기에 저장됩니다!
}

val_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# (데이터셋 클래스는 학습 때와 완벽히 동일)
class RSNAClfDataset(Dataset):
    def __init__(self, df, base_dir, transforms=None):
        self.df = df
        self.base_dir = base_dir
        self.transforms = transforms
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        
    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        full_path = os.path.join(self.base_dir, 'old_train', row['file_path'])
        try:
            with np.load(full_path) as loaded:
                img = loaded['data'].astype(np.float32) 
        except:
            img = np.zeros((3, 512, 512), dtype=np.float32)

        img_uint8 = (img * 255).astype(np.uint8)
        channels = [self.clahe.apply(img_uint8[i]) for i in range(3)]
        img_hwc = np.stack(channels, axis=-1) 
        
        if self.transforms: img_tensor = self.transforms(image=img_hwc)['image']
        else: img_tensor = torch.from_numpy(np.transpose(img_hwc, (2,0,1))).float() / 255.0

        return img_tensor, row['StudyInstanceUID'], row['slice_idx'], row['original_slice_number']

if __name__ == '__main__':
    print("📖 데이터 준비 중...")
    df_full = pd.read_csv(CONFIG['manifest_path'])
    df_golden = pd.read_csv(CONFIG['golden_bbox_csv'])
    golden_uids = set(df_golden['StudyInstanceUID'].astype(str).unique())
    df_full['StudyInstanceUID'] = df_full['StudyInstanceUID'].astype(str)
    
    # 평가용이므로 235명 환자의 '전체 슬라이스'를 대상으로 합니다!
    val_df = df_full[df_full['StudyInstanceUID'].isin(golden_uids)].reset_index(drop=True)
    
    val_ds = RSNAClfDataset(val_df, CONFIG['base_dir'], transforms=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=4)
    
    print("🛠️ 모델 로드 중...")
    model = timm.create_model(CONFIG['model_name'], pretrained=False, in_chans=3, num_classes=1)
    model.load_state_dict(torch.load(CONFIG['model_weight_path'], map_location=CONFIG['device']))
    model.to(CONFIG['device'])
    model.eval()
    
    results = []
    
    print("🚀 전체 슬라이스 확률 예측 시작!")
    with torch.no_grad():
        for images, uids, slice_idxs, orig_slice_nums in tqdm(val_loader, desc="Predicting"):
            images = images.to(CONFIG['device'])
            outputs = model(images)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            
            for i in range(len(probs)):
                results.append({
                    'StudyInstanceUID': uids[i],
                    'slice_idx': slice_idxs[i].item(),
                    'original_slice_number': orig_slice_nums[i].item(),
                    'pred_prob': probs[i]
                })
                
    pred_df = pd.DataFrame(results)
    pred_df.to_csv(CONFIG['output_csv'], index=False)
    print(f"✅ 예측 결과 저장 완료: {CONFIG['output_csv']}")

# cls * seg 확률

In [ ]:
import pandas as pd
import numpy as np

# --- 경로 설정 ---
CLF_PREDS_PATH = './runs/rsna_clf_efficientnet_golden_235/val_full_predictions.csv'

# 🚨 사용자님이 '확률화 시켜둔 마스크 CSV 파일'의 이름을 적어주세요!
SEG_RATIOS_PATH = r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\seg_bone_ratios_from_nifti.csv'

FINAL_SUBMISSION_PATH = './runs/rsna_clf_efficientnet_golden_235/final_patient_predictions.csv'

print("🔄 1. 데이터 불러오기...")
df_clf = pd.read_csv(CLF_PREDS_PATH)
df_seg = pd.read_csv(SEG_RATIOS_PATH)

# 🛡️ 데이터 타입 완벽 통일 (문자열 변환)
df_clf['StudyInstanceUID'] = df_clf['StudyInstanceUID'].astype(str)
df_seg['StudyInstanceUID'] = df_seg['StudyInstanceUID'].astype(str)

# [핵심] 결합할 기준키(Key) 타입 통일
# 💡 만약 df_seg에 'slice_idx'가 없고 'slice_number'만 있다면, 아래 merge_on 기준을 바꿔야 합니다.
merge_on_col = 'slice_idx' 
if 'slice_idx' not in df_seg.columns:
    print("⚠️ 마스크 데이터에 slice_idx가 없습니다! original_slice_number로 결합을 시도합니다.")
    merge_on_col = 'original_slice_number'
    # 이름 통일
    if 'slice_number' in df_seg.columns:
        df_seg = df_seg.rename(columns={'slice_number': 'original_slice_number'})

df_clf[merge_on_col] = df_clf[merge_on_col].astype(int)
df_seg[merge_on_col] = df_seg[merge_on_col].astype(int)

print(f"📊 분류 데이터: {len(df_clf):,}장 | 마스크 데이터: {len(df_seg):,}장")

print("🔄 2. 톱니바퀴 조립(Merge) 중...")
df_merged = pd.merge(df_clf, df_seg, on=['StudyInstanceUID', merge_on_col], how='inner')

print(f"✅ 병합 성공! 매칭된 슬라이스: {len(df_merged):,}장\n")

if len(df_merged) == 0:
    print("🚨 병합된 데이터가 0장입니다. UID 이름이나 슬라이스 번호 체계가 일치하는지 확인해야 합니다!")
else:
    # 3. 마법의 계산: 확률 × 비율
    for c in range(1, 8):
        # 마스크 컬럼 이름이 C1_ratio인지 C1인지 확인 후 계산
        ratio_col = f'C{c}_ratio' if f'C{c}_ratio' in df_merged.columns else f'C{c}'
        df_merged[f'C{c}_fracture_score'] = df_merged['pred_prob'] * df_merged[ratio_col]

    # 4. 환자별 최종 결론 내리기 (Max Pooling)
    print("🧠 3. 환자별 최종 점수(Max Pooling) 계산 중...")
    aggregation_dict = {f'C{c}_fracture_score': 'max' for c in range(1, 8)}
    df_patient = df_merged.groupby('StudyInstanceUID').agg(aggregation_dict).reset_index()

    df_patient = df_patient.rename(columns={
        'C1_fracture_score': 'C1', 'C2_fracture_score': 'C2',
        'C3_fracture_score': 'C3', 'C4_fracture_score': 'C4',
        'C5_fracture_score': 'C5', 'C6_fracture_score': 'C6', 'C7_fracture_score': 'C7',
    })

    # 전체 환자 골절 확률 도출 (C1~C7 점수 중 최고점)
    df_patient['patient_overall'] = df_patient[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].max(axis=1)
    df_patient = df_patient[['StudyInstanceUID', 'patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']]

    # 5. 저장 및 출력
    df_patient.to_csv(FINAL_SUBMISSION_PATH, index=False)
    print(f"✨ 최종 제출 파일 저장 완료! ➔ {FINAL_SUBMISSION_PATH}\n")
    
    # 결과 미리보기 출력
    print(df_patient.head().to_markdown(index=False))